In [2]:
#basics
import pandas as pd
import numpy as np

#sklearn
from sklearn.metrics import mean_absolute_error, mean_squared_error


# Import Data

In [3]:
!pwd


/home/alish/code/amelietatin/Predicting_land_cover/notebooks


In [4]:
df=pd.read_csv("/home/alish/code/amelietatin/Predicting_land_cover/data/final_data_2015_2035.csv")

df.drop("Unnamed: 0", axis=1, inplace=True)


# Assuming your DataFrame is named df
# Ensure 'quarter_start' column is in datetime format
df['quarter_start'] = pd.to_datetime(df['quarter_start'])

# Filter the DataFrame to include only rows with 'quarter_start' on or before 2024-01-01
df = df[df['quarter_start'] <= '2024-01-01']

# Optionally, reset the index if you want a clean DataFrame
df.reset_index(drop=True, inplace=True)

df

,SITECODE,quarter_start,Bare Ground,Built Area,Crops,Flooded Vegetation,Grass,Shrub and Scrub,Snow and Ice,Trees,Water,temperature_quarterly_mean,precipitation_quarterly_mean,water-vapor-pressure_quarterly_mean,cloud-cover_quarterly_mean
0,AT1301000,2015-07-01,0.004731,0.306967,9.740933,0.434059,3.750812,1.101940,0.000000,79.647261,4.151655,18.86,2.788798e-08,942.75470,0.536892
1,AT1302000,2015-07-01,0.000079,0.450186,1.870178,0.062068,2.485236,0.504686,0.000000,94.372208,0.105224,18.12,3.174909e-08,948.60770,0.553719
2,AT1304000,2015-07-01,0.000000,3.427528,67.818286,0.180076,1.440462,1.265232,0.000000,24.646051,0.000000,18.12,3.174909e-08,948.60770,0.553719
3,AT2112000,2015-07-01,3.268027,0.178362,0.008970,0.550204,0.366961,13.011084,0.232740,82.268944,0.000000,13.15,5.000581e-08,891.69970,0.644521
4,AT2116000,2015-07-01,0.000000,0.845768,8.414334,0.328737,22.908775,0.229168,0.000000,58.175196,7.122431,13.15,5.000581e-08,891.69970,0.644521
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21756,SE0820042,2024-01-01,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,96.878517,3.121345,0.000000,-6.88,1.913455e-08,469.72640,0.780085
21757,SE0820295,2024-01-01,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,99.995652,0.000000,0.000000,-10.24,2.840418e-08,411.43936,0.868867
21758,SE0820614,2024-01-01,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,99.961524,0.024216,0.000000,-7.09,1.898806e-08,455.15042,0.755212
21759,SE0820615,2024-01-01,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,99.986081,0.000929,0.000000,-7.23,1.825365e-08,453.09683,0.751676


# Basline Model: Persistence Model 

Assumes that the land cover in the next quarter will be the same as the current quarter. This is a very simple model and serves as a minimal benchmark.

## Preprocess the Data

In [5]:
data=df.copy()

In [6]:
# Sort the data by SITECODE and quarter_start
data.sort_values(by=['SITECODE', 'quarter_start'], inplace=True)

# Define feature and target columns
feature_cols = ['temperature_quarterly_mean', 'precipitation_quarterly_mean', 
                'water-vapor-pressure_quarterly_mean', 'cloud-cover_quarterly_mean']
target_cols = ['Bare Ground', 'Built Area', 'Crops', 'Flooded Vegetation', 'Grass', 
               'Shrub and Scrub', 'Snow and Ice', 'Trees', 'Water']

# Split the data into training and testing sets
train_df = data[data['quarter_start'] <= '2023-09-30']
test_df = data[data['quarter_start'] > '2023-09-30']

# Verify the splits
print("Training set size:", train_df.shape)
print("Testing set size:", test_df.shape)


Training set size: (20521, 15)
Testing set size: (1240, 15)


## Create the Baseline Model

In [7]:
test_df

,SITECODE,quarter_start,Bare Ground,Built Area,Crops,Flooded Vegetation,Grass,Shrub and Scrub,Snow and Ice,Trees,Water,temperature_quarterly_mean,precipitation_quarterly_mean,water-vapor-pressure_quarterly_mean,cloud-cover_quarterly_mean
20521,AT1301000,2023-10-01,0.042085,0.228138,7.159562,0.884416,4.705582,1.680005,0.000000,79.370999,4.050934,9.40,2.531495e-08,779.52980,0.700255
21141,AT1301000,2024-01-01,3.609320,0.636298,7.003258,1.828857,6.159897,2.985530,0.000000,69.628906,5.610794,4.79,6.275796e-09,623.55690,0.667620
20522,AT1302000,2023-10-01,0.000000,0.290207,0.003039,0.145998,3.323384,0.193750,0.000000,95.833727,0.187992,8.87,2.315852e-08,772.90753,0.687128
21142,AT1302000,2024-01-01,0.980301,0.904842,0.369008,2.407601,2.544262,5.885582,0.000000,85.920848,0.164198,4.50,6.374565e-09,620.18150,0.660555
20523,AT1304000,2023-10-01,0.000000,2.025102,28.311751,7.728438,24.433896,0.485758,0.000000,33.806562,0.000000,8.87,2.315852e-08,772.90753,0.687128
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21758,SE0820614,2024-01-01,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,99.961524,0.024216,0.000000,-7.09,1.898806e-08,455.15042,0.755212
21139,SE0820615,2023-10-01,0.000000,0.000000,0.016783,0.000000,0.000000,0.000000,99.132048,0.708938,0.000000,-6.39,1.587677e-08,477.30930,0.717153
21759,SE0820615,2024-01-01,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,99.986081,0.000929,0.000000,-7.23,1.825365e-08,453.09683,0.751676
21140,SE0820616,2023-10-01,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,99.985115,0.005196,0.000000,-7.38,1.571231e-08,466.49550,0.713865


In [8]:
# Create a dictionary to store the last known values from the training data
last_known_values = {}

# Populate the dictionary with the last known values for each SITECODE
for site in train_df['SITECODE'].unique():
    last_known_values[site] = train_df[train_df['SITECODE'] == site].iloc[-1]

# Create a DataFrame for the predictions
predictions = []

# Generate predictions using the last known values
for idx, row in test_df.iterrows():
    site = row['SITECODE']
    if site in last_known_values:
        last_values = last_known_values[site]
        predictions.append(last_values[target_cols].values)
    else:
        predictions.append([None] * len(target_cols))

predictions_df = pd.DataFrame(predictions, columns=target_cols, index=test_df.index)

# Drop rows where predictions contain None values (for sites not present in training data)
predictions_df.dropna(inplace=True)

# Align test_df with predictions_df by the index
aligned_test_df = test_df.loc[predictions_df.index]


## Evaluate the Baseline Model

In [9]:

# Initialize lists to store error values for each target
mae_values = []
rmse_values = []

# Calculate MAE and RMSE for each target
for target in target_cols:
    mae = mean_absolute_error(aligned_test_df[target], predictions_df[target])
    rmse = np.sqrt(mean_squared_error(aligned_test_df[target], predictions_df[target]))
    mae_values.append(mae)
    rmse_values.append(rmse)
    print(f'{target}: MAE = {mae}, RMSE = {rmse}')

# Optionally, you can print average MAE and RMSE across all targets
print(f'Average MAE: {np.mean(mae_values)}, Average RMSE: {np.mean(rmse_values)}')


Bare Ground: MAE = 2.1888484433815103, RMSE = 6.760082726642573
Built Area: MAE = 0.737789419511116, RMSE = 2.6613476244207948
Crops: MAE = 3.3394487690471535, RMSE = 7.440065429905238
Flooded Vegetation: MAE = 2.608078850222472, RMSE = 5.645735196512291
Grass: MAE = 4.626878157792852, RMSE = 8.782006469014151
Shrub and Scrub: MAE = 6.5913955193416, RMSE = 11.077483813678711
Snow and Ice: MAE = 23.043018652077674, RMSE = 43.475909254733544
Trees: MAE = 24.5274026420415, RMSE = 35.716029170145966
Water: MAE = 6.823726015759375, RMSE = 16.193525542611198
Average MAE: 8.276287385463917, Average RMSE: 15.305798358629387
